In [1]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook

random.seed(42)
import json
import os
import pandas as pd

datagenerator = DataGenerator()
utils = Utils()

2023-11-29 03:51:48.082350: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 03:51:48.121101: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
EXPERIMENTS_FILE = "experiments/noisy/004_irrelevant_anomaly_all_experiments.json"
with open(EXPERIMENTS_FILE) as file:
    experiments = json.load(file)
OUTPUT_FILE = EXPERIMENTS_FILE.replace("experiments.json", "results.json")

In [3]:
#EXPERIMENTS_FILE = "experiments/normal/005_init_lat_ind_best.json"
#with open(EXPERIMENTS_FILE) as file:
#    experiments = json.load(file)
#NOISE_RATIO = 0.50
#for experiment in experiments:
#    del experiment['auc_roc']
#    del experiment['auc_pr']
#    experiment['noise_type'] = "irrelevant_features"
#    experiment['noise_ratio'] =  NOISE_RATIO
#with open( f"experiments/noisy/003_irrelevant_features_{int(100*NOISE_RATIO)}.json", "w") as file:
#    json.dump(experiments, file)

In [4]:
# anomaly_type = "dependency"
# for experiment in experiments:
#    experiment["anomaly_type"] = anomaly_type
# with open(f"experiments/noisy/002_anomaly_study_{anomaly_type}.json", "w") as file:
#    json.dump(experiments,file)

In [5]:
#experiments = []
#path = "experiments/noisy/"
#for file in os.listdir(path):
#    if file.find("_irrelevant_features_") > -1:
#        print(path+file)
#        with open(path+file, "r") as file:
#            experiments += json.load(file)

In [6]:
# for experiment in experiments:
#    del experiment['auc_pr']
#    del experiment['auc_roc']

In [7]:
#with open(f"experiments/noisy/003_irrelevant_features_all_experiments.json", "w") as file:
#    json.dump(experiments,file)

In [8]:
#with open(f"experiments/noisy/003_irrelevant_features_all_experiments.json") as irr_file:
#    experiment_irr = json.load(irr_file)
#with open(f"experiments/noisy/002_anomaly_study_all_experiments.json") as ano_file:
#    experiment_ano = json.load(ano_file)

In [9]:
#experiments = experiment_irr +  experiment_ano

In [10]:
#with open(f"experiments/noisy/004_irrelevant_anomaly_all_experiments.json", "w") as file:
#    json.dump(experiments,file)

In [11]:
def create_anomaly_dataframe(dataset, anomaly_type=None, noise_type=None, noise_ratio=None):
    datagenerator.dataset = dataset
    data = datagenerator.generator(
        la=1.00,
        realistic_synthetic_mode=anomaly_type,
        stdscale=True,
        minmax=False,
        noise_type=noise_type,
        noise_ratio=noise_ratio
    )
    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)

    return Y_train, Y_test, lb_train, lb_test

In [ ]:
for experiment in tqdm_notebook(experiments):
    print(f"Start Experimento: {experiment}")
    if "auc_roc" not in experiment.keys() and "anomaly_type" in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_anomaly_dataframe(
            experiment["dataset"], 
            realistic_synthetic_mode=experiment["anomaly_type"]
        )
    if "auc_roc" not in experiment.keys() and "noise_type" in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_anomaly_dataframe(
            experiment["dataset"],
            noise_type=experiment["noise_type"],
            noise_ratio=experiment["noise_ratio"],
        )

        n_train = len(Y_train)
        data_dim = Y_train.shape[1]
        latent_dim = experiment["latent_dim"]
        nn_layers = tuple(map(int, experiment["layers"].split(",")))
        kernel = experiment["kernel"]
        n_inducing = experiment["n_inducing"]

        n_epochs = 1000
        lr = 0.01
        batch_size = 128

        try:
            gplvm = AD_GPLVM(
                latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size
            )
            gplvm.fit(Y_train)
            score = gplvm.predict_score(Y_test)
            metrics = utils.metric(y_true=lb_test, y_score=score)

            experiment["auc_roc"] = metrics["aucroc"]
            experiment["auc_pr"] = metrics["aucpr"]
        except:
            experiment["auc_roc"] = 0.00
            experiment["auc_pr"] = 0.00

        experiment["lr"] = lr
        experiment["batch_size"] = batch_size
        experiment["n_epochs"] = n_epochs

        experiment["train_noise"] = float(lb_train.sum() / len(Y_train))

        with open(OUTPUT_FILE, "w") as file:
            json.dump(experiments, file)

/tmp/ipykernel_28422/2407570982.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for experiment in tqdm_notebook(experiments):


  0%|          | 0/423 [00:00<?, ?it/s]

Start Experimento: {'dataset': '01_ALOI', 'kernel': 'rbf', 'layers': '5,5', 'latent_dim': 13, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 50, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01}
subsampling for dataset 01_ALOI...
current noise type: irrelevant_features
{'Samples': 10000, 'Features': 27, 'Anomalies': 302, 'Anomalies Ratio(%)': 3.02}
Kernel não escolhido!



  0%|                                                                                                                                                               | 0/1000 [00:00<?, ?it/s]
                                                                                                                                                                                             

Start Experimento: {'dataset': '02_annthyroid', 'kernel': 'rbf', 'layers': '10,10,10', 'latent_dim': 3, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 50, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01}
current noise type: irrelevant_features
{'Samples': 7200, 'Features': 6, 'Anomalies': 534, 'Anomalies Ratio(%)': 7.42}
Kernel não escolhido!


...


Loss: 200.66, iter no: 500: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:35<00:00, 27.73it/s]
                                                                                                                                                                                             

Start Experimento: {'dataset': '04_breastw', 'kernel': 'rbf', 'layers': '10,10,10', 'latent_dim': 4, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 50, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01}
generating duplicate samples for dataset 04_breastw...
current noise type: irrelevant_features
{'Samples': 1000, 'Features': 9, 'Anomalies': 360, 'Anomalies Ratio(%)': 36.0}
Kernel não escolhido!



Loss: 9.27, iter no: 500: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 112.40it/s]
                                                                                                                                                                                             

Start Experimento: {'dataset': '05_campaign', 'kernel': 'rbf', 'layers': '5,5', 'latent_dim': 15, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 100, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01}
subsampling for dataset 05_campaign...
current noise type: irrelevant_features
{'Samples': 10000, 'Features': 62, 'Anomalies': 1103, 'Anomalies Ratio(%)': 11.03}
Kernel não escolhido!



Loss: 181.49, iter no: 0:  10%|████████████▎                                                                                                              | 100/1000 [00:09<01:29, 10.04it/s]

In [12]:
pd.DataFrame(experiments).to_csv(OUTPUT_FILE.replace(".json", ".csv"))

NameError: name 'pd' is not defined